In [1]:
import pandas as pd
import re
import numpy as np
import csv
import spacy

In [2]:
file = pd.read_csv('result_800.csv')
file[0:3]

,text,link,name,description
0,is thinking that it is time for a Pimm's O'clo...,https://www.pinterest.co.uk/ickleali75/pimms-o...,The 16 best Pimms o'clock images on Pinterest ...,Hard cider gets a British makeover in this sum...
1,is thinking that it is time for a Pimm's O'clo...,https://www.pinterest.co.uk/mscookies/it-s-pim...,122 best It's Pimms O'Clock images on Pinteres...,"Explore Katherine Horejší's board ""It's Pimms ..."
2,is thinking that it is time for a Pimm's O'clo...,http://orangette.net/2010/07/thats-the-spirit/,That's the spirit | Orangetteorangette.net/201...,23 Jul 2010 - I get the sense that a lot of yo...


In [3]:
linkNames = file[['text','name']]
linkNames[0:5]

,text,name
0,is thinking that it is time for a Pimm's O'clo...,The 16 best Pimms o'clock images on Pinterest ...
1,is thinking that it is time for a Pimm's O'clo...,122 best It's Pimms O'Clock images on Pinteres...
2,is thinking that it is time for a Pimm's O'clo...,That's the spirit | Orangetteorangette.net/201...
3,is thinking that it is time for a Pimm's O'clo...,Pimm's O'Clock | MetaFilterwww.metafilter.com/...
4,is thinking that it is time for a Pimm's O'clo...,a pukka problem - The Account Planning Group C...


preprocess data: the preprocess here is just lower case 

In [4]:
#tokenize
def preprocess(sent):

    words = str(sent).lower().split()
    new_words = []
    for w in words:
        w = re.sub(r'[0-9]+', '', w)
        new_words.append(w)
        
    return ' '.join(new_words)

#groupby object, preporcess then count 
pro_text = linkNames['name'].apply(preprocess)
pro_text[0:3]

0    the  best pimms o'clock images on pinterest | ...
1     best it's pimms o'clock images on pinterest |...
2    that's the spirit | orangetteorangette.net///t...
Name: name, dtype: object

In [19]:
#number of search results
print(len(pro_text))
print(type(pro_text))

880
<class 'pandas.core.series.Series'>


1. Here we create a class to store each variable as an object 
2. Return the count of key words 'lyric, lyrics, quote, quotes, song, songs' in the name of the website, because the name of the website contains most of the information we need
3. Count the keywords and store it as Score. Score is a vector that contain keyword counts in each search result

In [5]:
class myQuote:
    def __init__(self, text):
        #read in CSV, process
        self.quoteText = text
        self.quoteID = hash(self.quoteText)
        self.link = []
        self.name = []
        self.description = []
        self.scores = []
    
    def __hash__(self):
        return self.quoteID
    
    def __str__(self):
        return "Object text: " + self.quoteText + '\n' +\
        "Links: " + str(self.link) + '\n' +\
        "Names: " + str(self.name) + '\n' +\
        "Description " + str(self.description) + '\n' +\
        "Scores: " + str(self.scores)



objects = {}
with open('result_800.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        texthash = hash(row['text'])
        if texthash not in objects:
            objects[texthash] = myQuote(row['text'])
        objects[texthash].link.append(row['link'])
        objects[texthash].name.append(row['name'])
        objects[texthash].description.append(row['description'])
        line = preprocess(row['name'])
        count = line.count("lyric") + line.count("lyrics") + line.count("quote") + line.count("quotes")
        objects[texthash].scores.append(count)
        #add extra fields in the class if I need to add more fields        
#print(len(objects.keys()))
#for item in objects.keys():
#    if sum(objects[item].scores) > 0:
#        print(objects[item])
        
def saveAsCSV(dictz,filename):
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
    writer.writerow(["hash"] + ["text"] + ["count"])
    for item in objects.keys():
        writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores])
    f.close()
    
saveAsCSV(objects, 'myprocessedquotes.csv')
    

select and return those has at least search results contain at least one keyword and save it as csv

In [9]:
def SearchResult1(results, filename):
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
    writer.writerow(["hash"] + ["text"] + ["count"]+ ["name"])
    for item in results:
        count = 0
        for score in objects[item].scores:
            if score > 0:
                count = count + 1
        if count > 2:
            writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores] + [objects[item].description] )
    f.close()

results = objects.keys()
SearchResult1(results, 'QuotesDetected.csv')

In [37]:
#remove punctionations
def preprocess2(sent):
    #remove punctustion
    sent = re.sub(r'[^\w\s]','',sent)
    words = sent.split()
    new_words = []
    for w in words:      
        new_words.append(w.lower())
        
    return ' '.join(new_words)

s = 'Well, here are some of the darkest jokes, found in the deep, disturbing...'
preprocess2(s)

'well here are some of the darkest jokes found in the deep disturbing'

1. select those entries has at least search results (link names) contain more than one keyword, return the score vectors of these entries (entry A). Unselected entries are labeled as NotQuote
2. Among entry (A), compute cosine similarity between posts and content (description) of the search result, those with 90% above similarity are labeled as quote. Those less than 90% are labeled as suspect. 

In [63]:
# use cosine similarity to compare text with search result
nlp = spacy.load('en')
def SearchResult2(results, filename):
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
    writer.writerow(["hash"] + ["text"] + ["count"]+ ["cosineSim"] +['label'])
    for item in results:
        count = 0
        for score in objects[item].scores:
            if score > 0:
                count = count + 1
        if count > 2:
            str1 = ''.join (objects[item].description)
            doc2 = nlp(preprocess2(str1))
            doc1 = nlp(preprocess2(objects[item].quoteText))
            if doc1.similarity(doc2) > 0.90:
                writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores] + [doc1.similarity(doc2)]+['quote'])
            else:
                writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores] + [doc1.similarity(doc2)]+['suspect'])
        else:
            writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + ['null'] + ['null']+['NotQuote'])
    f.close()
        
results = objects.keys()
SearchResult2(results, 'QuotesDetected3.csv')